In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import json

In [ ]:
train_file = "data.json"
with open(train_file, 'r') as json_file:
    data = json.load(json_file)

In [ ]:
def extract_entities(annotation):
    start = annotation['start']
    end = annotation['end']
    label = annotation['label'].upper()
    return start, end, label

def process_example(example):
    entities = [extract_entities(annotation) for annotation in example['annotations']]
    return {
        'text': example['text'],
        'entities': entities
    }

training_data = [process_example(example) for example in data]

In [ ]:
def process_training_example(nlp, training_example):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    entities = []

    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
          span1 = doc.char_span(start, end+1, label=label, alignment_mode="contract")
          if span1 is None:
            print(f"Skipping invalid entity: {label} [{start}:{end}]")
          else:
            entities.append(span1)
        else:
            entities.append(span)

    filtered_entities = spacy.util.filter_spans(entities)
    doc.ents = filtered_entities
    return doc

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
# Load a new spaCy model
nlp = spacy.blank("en")

# Create a DocBin to store processed documents
doc_bin = DocBin()

# Process training data and add to DocBin
for training_example in tqdm(training_data, desc="Processing training data"):
    processed_doc = process_training_example(nlp, training_example)
    doc_bin.add(processed_doc)

# Save the processed documents to disk
doc_bin.to_disk("train.spacy")

Processing training data: 100%|██████████| 80/80 [00:00<00:00, 519.22it/s]

Skipping invalid entity: ACTOR [273:277]
Skipping invalid entity: HASH_VALUES [197:261]
Skipping invalid entity: DOMAIN_NAME [188:193]
Skipping invalid entity: ACTOR [0:6]
Skipping invalid entity: URL [376:399]
Skipping invalid entity: URL [174:196]
Skipping invalid entity: URL [23:43]
Skipping invalid entity: DOMAIN_NAME [174:185]
Skipping invalid entity: DOMAIN_NAME [23:32]
Skipping invalid entity: IP_ADDRESSES [103:117]
Skipping invalid entity: IP_ADDRESSES [103:117]


In [ ]:
!python -m spacy init fill-config base_config.cfg base_config.cfg

2023-11-19 20:03:26.863899: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 20:03:26.863982: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-19 20:03:26.864030: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-19 20:03:28.180208: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train base_config.cfg --output ./output --paths.train /content/train.spacy --paths.dev /content/train.spacy

2023-11-19 20:03:38.587957: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 20:03:38.588023: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-19 20:03:38.588071: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-19 20:03:39.942678: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline ===

测试

In [ ]:
test = 'Synchronization can be enabled during an initial start and configuration of Password Manager or later. A Kaspersky Account will allow you accessing your online storage from different computers. If you have already worked with the component but haven’t enabled the sync yet, then enable it by clicking the Sync is disabled link and by clicking the Enable sync button in the pop-up window. The configuration wizard will either offer you to create a Kaspersky Account or to enter your credentials for an existing account. If you do not have a Kaspersky Account, create one following the recommendations of the wizard. A Kaspersky Account will allow you accessing your online storage from different computers. Once an account is activated, the Password Manager database will automatically synchronize with an online storage. Now when you have configured synchronization of Password Manager on all computers you can manage the passwords database and visit the sites for which you have added accounts from any computer without bothering to remember your passwords and their safe input. Any changes in your passwords database will appear on all synchronized computers automatically.'

In [ ]:
test[72:151]

'[HKEY_LOCAL_MACHINE]\nWkbpsevaXImgvkwkbpXSmj kswXGqvvajpRavwmkjXVqj=$SYS$DRV.EXE'

In [ ]:
len('www.darpa.mil/WorkArea/DownloadAsset.aspx?id=2147484449')

55

In [ ]:
import re
words_list= ['Kaspersky']

for word in words_list:
    for m in re.finditer(word,test):
        print (m.group(),m.start(), m.end())
        print(test[m.start():m.end()])

Kaspersky 105 114
Kaspersky
Kaspersky 447 456
Kaspersky
Kaspersky 540 549
Kaspersky
Kaspersky 617 626
Kaspersky


In [ ]:
b = 'On Sunday, Dec. 13, FireEye released information related to a breach and data exfiltration originating from an unknown actor FireEye is calling UNC2452. Unit 42 tracks this and related activity as the <actor>SolarStorm</actor>. According to FireEye, SolarStorm has compromised organizations across the globe via a supply chain attack that consists of a trojanized update file for the SolarWinds Orion Platform.'

In [ ]:
pattern = re.compile(r'<[^>]+>',re.S)
result = pattern.sub('', b)
print(result)

On Sunday, Dec. 13, FireEye released information related to a breach and data exfiltration originating from an unknown actor FireEye is calling UNC2452. Unit 42 tracks this and related activity as the SolarStorm. According to FireEye, SolarStorm has compromised organizations across the globe via a supply chain attack that consists of a trojanized update file for the SolarWinds Orion Platform.


数据处理

In [ ]:
label = ['MALWARE_NAME','MALWARE_NAME','MALWARE_NAME','MALWARE_NAME','GEO_LOCATION','GEO_LOCATION']

In [ ]:
rawText = ['You already know how to easily create and use the passwords database using Password Manager in <malware> Kaspersky PURE 3.0 <malware> for quick and safe on-site authorization.'
,'But keeping pace with time we all use several computers (both stand-alone PCs and notebooks) in several places, e.g. at home and at work. And the reasonable question is how to make the created passwords database available and augmented on all computers you work on? To bridge the gap a new feature was introduced in <malware> Kaspersky PURE 3.0 <malware> that allows synchronizing databases of Password Manager.'
,'Thus, with <malware> Kaspersky PURE 3.0 <malware> installed and the Internet available, you can use your passwords database on all computers of yours in any part of the world.'
,'Synchronization can be enabled during an initial start and configuration of Password Manager or later. A Kaspersky Account will allow you accessing your online storage from different computers. If you have already worked with the component but haven’t enabled the sync yet, then enable it by clicking the Sync is disabled link and by clicking the Enable sync button in the pop-up window. The configuration wizard will either offer you to create a Kaspersky Account or to enter your credentials for an existing account. If you do not have a Kaspersky Account, create one following the recommendations of the wizard. A Kaspersky Account will allow you accessing your online storage from different computers. Once an account is activated, the Password Manager database will automatically synchronize with an online storage. Now when you have configured synchronization of Password Manager on all computers you can manage the passwords database and visit the sites for which you have added accounts from any computer without bothering to remember your passwords and their safe input. Any changes in your passwords database will appear on all synchronized computers automatically.'
,'But keeping pace with time we all use several computers (both stand-alone PCs and notebooks) in several places, e.g. at <geo_loc> home <geo_loc> and at <geo_loc> work <geo_loc>. And the reasonable question is how to make the created passwords database available and augmented on all computers you work on?'
,'Thus, with Kaspersky PURE 3.0 installed and the Internet available, you can use your passwords database on all computers of yours in any <geo_loc> part of the world <geo_loc>.'


           ]

In [ ]:
entities = ['Kaspersky PURE 3.0','Kaspersky PURE 3.0','Kaspersky PURE 3.0','Kaspersky',['home', 'work'],'part of the world']

In [ ]:
resultText = []
for f in range(len(rawText)):
  pattern = re.compile(r'<[^>]+>',re.S)
  result = pattern.sub('', rawText[f])
  resultText.append(result)
print(resultText)

['You already know how to easily create and use the passwords database using Password Manager in  Kaspersky PURE 3.0  for quick and safe on-site authorization.', 'But keeping pace with time we all use several computers (both stand-alone PCs and notebooks) in several places, e.g. at home and at work. And the reasonable question is how to make the created passwords database available and augmented on all computers you work on? To bridge the gap a new feature was introduced in  Kaspersky PURE 3.0  that allows synchronizing databases of Password Manager.', 'Thus, with  Kaspersky PURE 3.0  installed and the Internet available, you can use your passwords database on all computers of yours in any part of the world.', 'Synchronization can be enabled during an initial start and configuration of Password Manager or later. A Kaspersky Account will allow you accessing your online storage from different computers. If you have already worked with the component but haven’t enabled the sync yet, then 

In [ ]:
procesData = []
for i in range(len(label)):
  dic = {"text": resultText[i],"annotations":[]}
  if type(entities[i]) == list:
    words_list = entities[i]
    for word in words_list:
      for m in re.finditer(word,resultText[i]):
        dic['annotations'].append({"start": m.start(), "end": m.end(), "label": label[i], "entity": word})
  else:
    words = entities[i]
    for m in re.finditer(words,resultText[i]):
      dic['annotations'].append({"start": m.start(), "end": m.end(), "label": label[i], "entity": words})
  procesData.append(dic)

In [ ]:
json.dumps(procesData)

'[{"text": "You already know how to easily create and use the passwords database using Password Manager in  Kaspersky PURE 3.0  for quick and safe on-site authorization.", "annotations": [{"start": 96, "end": 114, "label": "MALWARE_NAME", "entity": "Kaspersky PURE 3.0"}]}, {"text": "But keeping pace with time we all use several computers (both stand-alone PCs and notebooks) in several places, e.g. at home and at work. And the reasonable question is how to make the created passwords database available and augmented on all computers you work on? To bridge the gap a new feature was introduced in  Kaspersky PURE 3.0  that allows synchronizing databases of Password Manager.", "annotations": [{"start": 317, "end": 335, "label": "MALWARE_NAME", "entity": "Kaspersky PURE 3.0"}]}, {"text": "Thus, with  Kaspersky PURE 3.0  installed and the Internet available, you can use your passwords database on all computers of yours in any part of the world.", "annotations": [{"start": 12, "end": 30, "label